In [ ]:
!git clone https://github.com/axelcarlier/animals.git

In [ ]:
import PIL
from PIL import Image
import numpy as np
import os

# Chemin vers la base de données
path = "./animals/unlabelled/"
# Indice d'ajout de données dans les variables x et y 
i = 0
# Préparation des structures de données pour x et y
x = np.zeros((50000, 64, 64, 3))

# Parcours des fichiers (classés) du répertoire
dirs = os.listdir(path)

for item in dirs:
  # Image : on va remplir la variable x
  # Lecture de l'image
  img = Image.open(path + item)
  # Remplissage de la variable x
  x[i] = np.asarray(img)
  i = i+1

x = x.astype('uint8')

In [ ]:
# Visualiser la base de données
import matplotlib.pyplot as plt
from PIL import Image

fig = plt.figure(figsize=(15, 10))

for i in range(20):
  num = np.random.randint(x.shape[0])
  plt.subplot(4, 5, i+1)
  plt.imshow(x[num]);
  plt.axis('off')
plt.show()
